# Final Project KK : Implementasi Artikel

Anggota Kelompok 1:
1. Maheswara Danendra Satriananda (5025201060)
2. Rere Arga Dewanata (5025201078)
3. Andhika Ditya Bagaskara D. (5025201096)
4. Naufal Faadhilah (5025201221)

Artikel Referensi:  
A Powerful Genetic Algorithm for Traveling Salesman Problem

Dataset:  
USA Latitude Longitude States
([LinkDataset](https://www.kaggle.com/datasets/washimahmed/usa-latlong-for-state-abbreviations?resource=download))



## Tahap Persiapan Dataset

In [ ]:
# Import library yang dibutuhkan
import pandas as pd
import numpy as np
import random
import operator
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Melakukan pembacaan dataset
path_state = "/content/US-States.csv"
df_state = pd.read_csv(path_state)
df_state.head()

In [ ]:
# Mengubah Longitude dan Latitude menjadi format sumbu kartesian (x,y)
# x = RadiusBumi * cos( radiance(lat) ) * cos(radiance (lon))
# y = RadiusBumi * cos( radiance(lat) ) * sin(radiance (lon))
from math import radians,cos,sin

EARTH_RADIUS = 6371
lat = df_state["Latitude"].map(radians)
lon = df_state["Longitude"].map(radians)
x   = lon.map(cos)*lat.map(cos)*EARTH_RADIUS
y   = lon.map(cos)*lat.map(sin)*EARTH_RADIUS

df_state["lat_radians"] = lat
df_state["lon_radians"] = lon
df_state["x"] = x
df_state["y"] = y
df_state.head()

In [ ]:
# Mendapatkan informasi general dataset
df_state.info()

In [ ]:
# Mengecek apabila terdapat data yang "null" pada setiap kolom
df_state.isnull().sum()

In [ ]:
# Mengecek apabila terdapat baris yang duplikat 
df_state.duplicated().sum()

In [ ]:
# Membuat dataframe baru yang hanya berisi kolom x dan y
df_statepos = df_state.drop(["State", "Latitude", "Longitude", "City","lat_radians", "lon_radians"], 1)
df_statepos.head()

In [ ]:
# Fungsi untuk menghitung jarak antara 2 states
def distance_between_states(states):
    data = dict()
    for index, value in enumerate(states):
        x1 = states[index][0]
        y1 = states[index][1]
        if index + 1 <= len(states)-1:
            x2 = states[index+1][0]
            y2 = states[index+1][1]
            xdiff = x2 - x1
            ydiff = y2 - y1
            dst = (xdiff*xdiff + ydiff*ydiff)** 0.5
            data['Distance from state '+ str(index+1) +' to state ' + str(index+2)] = dst 
        elif index + 1 > len(states)-1:
            x2 = states[0][0]
            y2 = states[0][1]
            xdiff = x2 - x1
            ydiff = y2 - y1
            dst = (xdiff*xdiff + ydiff*ydiff)** 0.5
            data['Distance from state '+ str(index+1) + ' to state ' + str(index +2 -len(states))] = dst
    print(data)          
    return data

In [ ]:
# Fungsi untuk menghitung total jarak yang dibutuhkan
def total_distance(states):
    total = sum(distance_between_states(states).values())
    return total

In [ ]:
# Mengubah dataframe menjadi sebuah list
state_list = df_statepos.values.tolist()
print(state_list)

In [ ]:
# Menghitung jarak antara dua state
val = distance_between_states(state_list).values()
print(val)

In [ ]:
# Menghitung jarak total 
total_distance(state_list)

In [ ]:
# Fungsi untuk menghasilkan path random 
def generatePath(states):
    path = random.sample(states, len(states))
    return path

In [ ]:
# Membuat path random 
list = generatePath(state_list)
print(list)

In [ ]:
# Melakukan visualisasi plotting jarak random pada setiap state
state_names = df_state["State"]
def plot_pop(states):
    plt.figure(figsize=(20,10))
    x = [i[0] for i in states]
    y = [i[1] for i in states]
    x1=[x[0],x[-1]]
    y1=[y[0],y[-1]]
    plt.plot(x, y, 'b', x1, y1, 'b')
    plt.scatter (x, y)
    j = df_statepos["x"]
    k = df_statepos["y"]
    
    for i, txt in enumerate(state_names):
        plt.annotate(txt, (j[i], k[i]),horizontalalignment='center', verticalalignment='bottom')
    plt.show()
    return
plot_pop(list) 

In [ ]:
# Menghasilkan Populasi awal
def initialPopulation(states, populationSize):
    population = [generatePath(states) for i in range(0, populationSize)]
    return population

n_population = 10
population = initialPopulation(state_list,n_population)
print(population)

In [ ]:
for idx, pop_plot in enumerate (population):
    print('Initial Population '+ str(idx),pop_plot)

In [ ]:
# Visualisasi pada populasi awal
for pop_plot in population:
    plot_pop(pop_plot)

In [ ]:
# Menentukan nilai path fitness
def path_fitness(states):
    total_dis = total_distance(states)
    fitness= 0.0
    if fitness == 0:
        fitness = 1 / float(total_dis)
    return fitness
path_fitness(state_list)

In [ ]:
# Mengurutkan fitness path dari yang terbesar
def rankPathes(population):
    fitnessResults = {}
    for i in range(len(population)):
        fitnessResults[i] = path_fitness(population[i])
        
    return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)
rankPathes(population)

In [ ]:
# Membuat fungsi untuk seleksi
def perform_selection(pop, eliteSize):
    df = pd.DataFrame(np.array(pop), columns=["Index","Fitness"])
    df['cumulative_sum'] = df.Fitness.cumsum()
    df['cumulative_percentage'] = 100*df.cumulative_sum/df.Fitness.sum()
    selected_values = [pop[i][0] for i in range(eliteSize)]
    
    for i in range(len(pop) - eliteSize):
        pick = 100*random.random()
        for i in range(0, len(pop)):
            if pick <= df.iat[i,3]:
                selected_values.append(pop[i][0])
                break
                
    return selected_values

In [ ]:
out11 = rankPathes(population)
selected_values = perform_selection(out11,5)
print(selected_values)

In [ ]:
def do_mating_pool(population, selected_values):
    matingpool = [population[selected_values[i]] for i in range(len(selected_values))]
    return matingpool
mp = do_mating_pool(population, selected_values)

In [ ]:
# Membuat fungsi untuk melakukan perkawinan
def do_breed(first_parent, second_parent):
    generation_1= int(random.random() * len(first_parent))
    generation_2 = int(random.random() * len(second_parent))
    
    first_generation = min(generation_1, generation_2)
    last_generation = max(generation_1, generation_2)

    tot_parent1 = [first_parent[i] for i in range(first_generation, last_generation)]
    tot_parent2 = [i for i in second_parent if i not in tot_parent1]

    tot = tot_parent1 + tot_parent2
    return tot

In [ ]:
# Melakukan perkawinan pada populasi
def do_breed_population(my_mating_pool, eliteSize):
    ln = len(my_mating_pool) - eliteSize
    pl = random.sample(my_mating_pool, len(my_mating_pool))
    tot1 = [my_mating_pool[i] for i in range(eliteSize)]
    tot2 = [do_breed(pl[i], pl[len(my_mating_pool)-i-1]) for i in range(ln)]
    tot = tot1+tot2
    return tot
do_breed_population(mp,2)

In [ ]:
# Membuat fungsi untuk mutation
def do_mutation(indiv, mutat_rate):
    for exchanged in range(len(indiv)):
        if(random.random() < mutat_rate):
            exchanged_with = int(random.random() * len(indiv))
            
            city1 = indiv[exchanged]
            city2 = indiv[exchanged_with]
            
            indiv[exchanged] = city2
            indiv[exchanged_with] = city1
    return indiv

In [ ]:
# membuat fungsi mutation population
def do_mutation_pop(population, mutat_rate):
    mutated_population = [do_mutation(population[i], mutat_rate) for i in range(len(population))]
    return mutated_population

mutation_rate = 0.1   
do_mutation_pop(population, mutation_rate)

In [ ]:
# Mendapatkan gen yang berikutnya
def get_following_gen(existing_gen, eliteSize, mutat_rate):
    pop = rankPathes(existing_gen)
    selected_values = perform_selection(pop, eliteSize)
    my_mating_pool = do_mating_pool(existing_gen, selected_values)
    tot = do_breed_population(my_mating_pool, eliteSize)
    following_gen = do_mutation(tot, mutat_rate)
    return following_gen
get_following_gen(population, 5, mutation_rate)

In [ ]:
def get_names(result_lst, states, name_lst):
    names = []
    for index,value in enumerate(result_lst):
        for i,v in enumerate(states):
            if value == v:
                names.append(name_lst[i])
    return names

In [ ]:
def GA(state_names,states, population_size, eliteSize, mutat_rate, generations):
    population = initialPopulation(states,population_size)
    print("Incipient distance: " + str(1 / rankPathes(population)[0][1]))
    for i in range(generations):
        population = get_following_gen(population, eliteSize, mutat_rate)
    
    print("Eventual distance: " + str(1 / rankPathes(population)[0][1]))
    optimal_route_id = rankPathes(population)[0][0]
    optimal_route = population[optimal_route_id]
    ordered_states = get_names(optimal_route,states,state_names)
    print([(indx,val) for indx,val in enumerate(ordered_states)])
    plot_pop(optimal_route)
    return optimal_route

result_lst = GA(state_names,state_list, population_size=100, 
                 eliteSize=5, mutat_rate=0.1, 
                 generations=500)